In [102]:
import os
import pandas as pd
import numpy as np


In [121]:
obs_df = pd.read_csv(os.path.join("data", "db", "Observations.csv"))

In [122]:
obs_df.head()


,Player Id,Salary Year,Annual Salary,Advanced Pitching_BABIP.Year,Advanced Pitching_BABIP.Year-1,Advanced Pitching_BB/9.Year,Advanced Pitching_BB/9.Year-1,Advanced Pitching_BQR.Year,Advanced Pitching_BQR.Year-1,Advanced Pitching_BQR_S.Year,...,Pitching_SO.Year,Pitching_SO.Year-1,Pitching_SV.Year,Pitching_SV.Year-1,Pitching_SVO.Year,Pitching_SVO.Year-1,Pitching_W.Year,Pitching_W.Year-1,Pitching_WHIP.Year,Pitching_WHIP.Year-1
0,ygallardo,2015,"$ 13,000,000",0.303,0.298,3.32,2.53,26.0,10.0,7.0,...,121.0,146.0,0.0,0.0,0.0,0.0,13.0,8.0,1.42,1.29
1,jshields,2015,"$ 18,750,000",0.303,0.296,3.60,1.74,16.0,13.0,7.0,...,216.0,180.0,0.0,0.0,0.0,0.0,13.0,14.0,1.33,1.18
2,mbetts,2015,"$ 514,500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,bposey,2015,"$ 18,555,555",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,jquintana,2015,"$ 4,200,000",0.330,0.320,1.92,2.34,12.0,17.0,4.0,...,177.0,178.0,0.0,0.0,0.0,0.0,9.0,9.0,1.27,1.24


In [123]:
# Cleanup
# Replace mising values
obs_df = obs_df.fillna(0.0)
obs_df = obs_df.replace('-', 0.0)
obs_df = obs_df.replace('', 0.0)
obs_df = obs_df.replace('.---', 0.0)
obs_df = obs_df.replace('nan', 0.0)

#obs_df = obs_df.round(
obs_df.round(3)
# Convert Annual Salary to numeric
obs_df['Annual Salary'] = obs_df['Annual Salary'].str.replace("$", "")
obs_df['Annual Salary'] = obs_df['Annual Salary'].str.replace(",", "")
obs_df['Annual Salary'] = pd.to_numeric(obs_df['Annual Salary'])
obs_df.head()


,Player Id,Salary Year,Annual Salary,Advanced Pitching_BABIP.Year,Advanced Pitching_BABIP.Year-1,Advanced Pitching_BB/9.Year,Advanced Pitching_BB/9.Year-1,Advanced Pitching_BQR.Year,Advanced Pitching_BQR.Year-1,Advanced Pitching_BQR_S.Year,...,Pitching_SO.Year,Pitching_SO.Year-1,Pitching_SV.Year,Pitching_SV.Year-1,Pitching_SVO.Year,Pitching_SVO.Year-1,Pitching_W.Year,Pitching_W.Year-1,Pitching_WHIP.Year,Pitching_WHIP.Year-1
0,ygallardo,2015,13000000,0.303,0.298,3.32,2.53,26.0,10.0,7.0,...,121.0,146.0,0.0,0.0,0.0,0.0,13.0,8.0,1.42,1.29
1,jshields,2015,18750000,0.303,0.296,3.60,1.74,16.0,13.0,7.0,...,216.0,180.0,0.0,0.0,0.0,0.0,13.0,14.0,1.33,1.18
2,mbetts,2015,514500,0.000,0.000,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
3,bposey,2015,18555555,0.000,0.000,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
4,jquintana,2015,4200000,0.330,0.320,1.92,2.34,12.0,17.0,4.0,...,177.0,178.0,0.0,0.0,0.0,0.0,9.0,9.0,1.27,1.24


In [124]:
obs_df.head()

,Player Id,Salary Year,Annual Salary,Advanced Pitching_BABIP.Year,Advanced Pitching_BABIP.Year-1,Advanced Pitching_BB/9.Year,Advanced Pitching_BB/9.Year-1,Advanced Pitching_BQR.Year,Advanced Pitching_BQR.Year-1,Advanced Pitching_BQR_S.Year,...,Pitching_SO.Year,Pitching_SO.Year-1,Pitching_SV.Year,Pitching_SV.Year-1,Pitching_SVO.Year,Pitching_SVO.Year-1,Pitching_W.Year,Pitching_W.Year-1,Pitching_WHIP.Year,Pitching_WHIP.Year-1
0,ygallardo,2015,13000000,0.303,0.298,3.32,2.53,26.0,10.0,7.0,...,121.0,146.0,0.0,0.0,0.0,0.0,13.0,8.0,1.42,1.29
1,jshields,2015,18750000,0.303,0.296,3.60,1.74,16.0,13.0,7.0,...,216.0,180.0,0.0,0.0,0.0,0.0,13.0,14.0,1.33,1.18
2,mbetts,2015,514500,0.000,0.000,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
3,bposey,2015,18555555,0.000,0.000,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
4,jquintana,2015,4200000,0.330,0.320,1.92,2.34,12.0,17.0,4.0,...,177.0,178.0,0.0,0.0,0.0,0.0,9.0,9.0,1.27,1.24


# Feature Selection


In [107]:
# Calculate ratio of salary to avg. salary
##salary_year_groups = obs_df.groupby(['Salary Year'])
#salary_year_groups['Annual Salary']

# TODO -- Right now we only have one year and I can't seem to get this working, so I'll punt for now.

In [125]:
# Replace categorical variables with dummy variables
import re
#fielding_pos_columns = [col for col in list(obs_df.columns) if re.search(r'Fielding_POS', col)]   
col = 'Fielding_POS.Year'
dummy_vars_df = pd.get_dummies(obs_df[col])
for dummy_col in dummy_vars_df.columns:
    obs_df[dummy_col] = dummy_vars_df[dummy_col]
    


In [126]:
# Select subset of features
# This is me making an educated guess about which ones are predictive. I'll just use the most common stats
# for now, but will add more later as they become available (and as the amount of data available grows).

obs_df.head()

,Player Id,Salary Year,Annual Salary,Advanced Pitching_BABIP.Year,Advanced Pitching_BABIP.Year-1,Advanced Pitching_BB/9.Year,Advanced Pitching_BB/9.Year-1,Advanced Pitching_BQR.Year,Advanced Pitching_BQR.Year-1,Advanced Pitching_BQR_S.Year,...,Pitching_WHIP.Year,Pitching_WHIP.Year-1,0.0,1B,2B,3B,C,OF,P,SS
0,ygallardo,2015,13000000,0.303,0.298,3.32,2.53,26.0,10.0,7.0,...,1.42,1.29,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,jshields,2015,18750000,0.303,0.296,3.60,1.74,16.0,13.0,7.0,...,1.33,1.18,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,mbetts,2015,514500,0.000,0.000,0.00,0.00,0.0,0.0,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,bposey,2015,18555555,0.000,0.000,0.00,0.00,0.0,0.0,0.0,...,0.00,0.00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,jquintana,2015,4200000,0.330,0.320,1.92,2.34,12.0,17.0,4.0,...,1.27,1.24,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [127]:
len(obs_df)
print(obs_df['Fielding_POS.Year'])

0       P
1       P
2      OF
3      1B
4       P
5      SS
6      SS
7      OF
8       P
9      1B
10     OF
11     OF
12     2B
13     SS
14      P
15      P
16     SS
17      P
18     3B
19     OF
20      P
21     SS
22      P
23     3B
24     OF
25     OF
26     2B
27     OF
28     OF
29     OF
       ..
223    OF
224    1B
225    SS
226    SS
227    OF
228    2B
229    OF
230    OF
231     P
232    2B
233     P
234    OF
235    SS
236    3B
237     C
238    OF
239    1B
240    SS
241    2B
242     P
243    OF
244    1B
245    1B
246     C
247     P
248     C
249     P
250    1B
251     P
252     P
Name: Fielding_POS.Year, dtype: object


In [128]:
columns = ['Player Id', 'Salary Year', 'Annual Salary']
stats = ['Batting_G', 'Batting_AVG', 'Batting_OBP', 'Batting_SLG', 'Batting_RBI',
         'Pitching_G', 'Pitching_ERA', 'Pitching_WHIP', 'Pitching_IP',
         'Fielding_G', 'Fielding_FPCT', 'Fielding_INN']
years = ['Year', 'Year-1']

stat_columns = []
for stat in stats:
    for year in years:
        stat_columns.append("{}.{}".format(stat, year))
        
columns.extend(stat_columns)
columns.extend(dummy_vars_df.columns)
#print(columns)
#print(str(len(columns)) + " columns")
#for col in obs_df.columns.values:
#    print('Batting_AVG' in col or 'Pitching_ERA' in col)

obs_df = pd.DataFrame(obs_df, columns=columns)
obs_df.to_csv("observations-cleaned.csv")
obs_df.head()

,Player Id,Salary Year,Annual Salary,Batting_G.Year,Batting_G.Year-1,Batting_AVG.Year,Batting_AVG.Year-1,Batting_OBP.Year,Batting_OBP.Year-1,Batting_SLG.Year,...,Fielding_INN.Year,Fielding_INN.Year-1,0.0,1B,2B,3B,C,OF,P,SS
0,ygallardo,2015,13000000,2.0,31.0,0.500,0.111,0.500,0.127,0.750,...,184.1,192.1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,jshields,2015,18750000,34.0,3.0,0.132,0.286,0.145,0.286,0.162,...,202.1,227.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,mbetts,2015,514500,145.0,52.0,0.291,0.291,0.341,0.368,0.479,...,1254.0,326.2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,bposey,2015,18555555,150.0,147.0,0.318,0.311,0.379,0.364,0.470,...,323.2,261.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,jquintana,2015,4200000,4.0,4.0,0.000,0.000,0.000,0.000,0.000,...,206.1,200.1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [136]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
X = np.asarray(obs_df[obs_df.columns[3:]])
Y = np.asarray(obs_df['Annual Salary'])

from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
regr.fit(X_train, Y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [131]:
regr.score(X_train, Y_train)

0.32716198408705843

In [133]:
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(regr, X, Y, cv=5)

In [134]:
scores

array([-0.24345864,  0.22711965, -0.68248169,  0.02945826, -0.28999707])

In [135]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: -0.19 (+/- 0.62)
